In [75]:
import pandas as pd
import numpy as np

In [76]:
data=pd.read_excel('result.xlsx')[0:32]
data

,Unnamed: 0,acid_concentration/M,metal_concentration/M,depense_rate,Unnamed: 4,phase/ 0 alpha/1 beta/2 mix,Unnamed: 6,Ibeta,Ialpha,Ialpha/Itotal
0,1,0.40,0.40,1.00,NaN,2,NaN,307.20,648.95,0.678714
1,2,0.40,0.20,1.00,NaN,2,NaN,2031.05,355.97,0.149126
2,3,0.40,0.05,1.00,NaN,1,NaN,NaN,NaN,0.000000
3,4,0.20,0.40,1.00,NaN,2,NaN,1090.34,821.90,0.429811
4,5,0.20,0.20,1.00,NaN,2,NaN,1980.52,379.70,0.160875
5,6,0.20,0.05,1.00,NaN,1,NaN,NaN,NaN,0.000000
6,7,0.05,0.40,1.00,NaN,1,NaN,NaN,NaN,0.000000
7,8,0.05,0.20,1.00,NaN,1,NaN,NaN,NaN,0.000000
8,9,0.05,0.05,1.00,NaN,1,NaN,NaN,NaN,0.000000
9,10,0.40,0.40,0.20,NaN,2,NaN,4449.72,452.86,0.092372


In [77]:
X=data.iloc[:,1:4]

In [78]:
X=np.array(X)

In [79]:
len(X)

32

In [80]:
y=data[['phase/ 0 alpha/1 beta/2 mix']]

In [81]:
y=np.array(y).ravel()

In [82]:
y

array([2, 2, 1, 2, 2, 1, 1, 1, 1, 2, 2, 1, 2, 2, 1, 1, 1, 1, 2, 2, 1, 2,
       2, 1, 1, 1, 1, 0, 0, 0, 0, 0], dtype=int64)

In [83]:
from sklearn.model_selection import KFold
import torch
import torch.nn as nn
import torch.optim as optim
from scipy.stats import entropy
import torchbnn as bnn
from sklearn.preprocessing import MinMaxScaler

In [84]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
   


kf = KFold(n_splits=5, shuffle=True, random_state=42)
cv_accuracies = []

for train_index, val_index in kf.split(X):
   
    X_train_np, X_val_np = X[train_index], X[val_index]
    y_train_np, y_val_np = y[train_index], y[val_index]

    
    scaler = MinMaxScaler(feature_range=(0, 1))  
    X_train_scaled = scaler.fit_transform(X_train_np)
    X_val_scaled   = scaler.transform(X_val_np)

   
    X_train_t = torch.tensor(X_train_scaled, dtype=torch.float32, device=device)
    y_train_t = torch.tensor(y_train_np, dtype=torch.long, device=device)
    X_val_t   = torch.tensor(X_val_scaled, dtype=torch.float32, device=device)
    y_val_t   = torch.tensor(y_val_np, dtype=torch.long, device=device)

    
    model = nn.Sequential(
        bnn.BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=X_train_t.shape[1], out_features=50),
        nn.ReLU(),
        bnn.BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=50, out_features=len(np.unique(y)))
    ).to(device)

    criterion = nn.CrossEntropyLoss()
    kl_loss = bnn.BKLLoss(reduction='mean')
    optimizer = optim.Adam(model.parameters(), lr=1e-3)

    for epoch in range(500):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train_t)
        loss = criterion(outputs, y_train_t) + 0.1 * kl_loss(model)
        loss.backward()
        optimizer.step()

  
    model.eval()
    with torch.no_grad():
        val_preds = model(X_val_t).argmax(dim=1)
        acc = (val_preds == y_val_t).float().mean().item()
        
        cv_accuracies.append(acc)

mean_acc = np.mean(cv_accuracies)
std_cv_acc = np.std(cv_accuracies)  
sem_cv_acc = std_cv_acc / np.sqrt(len(cv_accuracies))

print("Mean Accuracy:", mean_acc)
print("Accuracy SEM:", sem_cv_acc)


Mean Accuracy: 0.6190476357936859
Accuracy SEM: 0.06459362030976286
